# Spillover effect measures for ESG groups

<div class="alert alert-info">
使用分位数来进行ESG分组，然后构建每组的ESG指数，再进行波动率溢出效应的度量，构建波动溢出网络。
    
步骤：
1. 将股票和ESG数据都做成面板数据（季度）
2. 
    
Draft: 2024.9.1 Updated:2024.9.4 Version: 0.0.3
</div>


## Processing

### Price data

价格数据的频率为日频，2212个股票

In [54]:
import pandas as pd
import numpy as np
df_prices = pd.read_csv('datasets/processed_data/close.csv')
df_prices.index = pd.to_datetime(df_prices['time'])
df_prices = df_prices.iloc[:, 1:]
print(df_prices.shape)
df_prices.head(1)

(3504, 2212)


,000001,000002,000004,000005,000006,000007,000008,000009,000010,000011,...,601933,601939,601958,601988,601989,601991,601992,601996,601998,601999
time,,,,,,,,,,,,,,,,,,,,,
2010-01-04,926.73,1091.9,NaN,61.24,158.64,55.68,NaN,47.34,39.24,35.42,...,NaN,6.42,23.12,4.56,7.63,18.49,NaN,NaN,8.26,11.42


### ESG data

In [2]:
import pandas as pd
path='datasets/华证2009-2023年（含细分项+季度)）/华证esg评级2009-2023（细分项）/华证esg评级含细分项（季度）2009-2023.xlsx'
df_esg = pd.read_excel(path, dtype={'证券代码':str})


,证券代码,年份,证券简称,评级日期,综合评级,综合得分,E评级,E得分,S评级,S得分,G评级,G得分,证监会行业新,同花顺行业新,申万行业
0,1,2009,平安银行,1/31/2009,BBB,82.56,B,72.98,BBB,80.26,A,88.73,货币金融服务,股份制银行,股份制银行Ⅲ


1. 处理证券代码不全；
2. 处理评级日期格式为datetime格式

In [24]:
df_esg['证券代码'] = df_esg['证券代码'].str.zfill(6).astype(str)
df_esg['评级日期'] = pd.to_datetime(df_esg['评级日期'])
df_esg.index = df_esg['评级日期']
df_esg.head(1)

,证券代码,年份,证券简称,评级日期,综合评级,综合得分,E评级,E得分,S评级,S得分,G评级,G得分,证监会行业新,同花顺行业新,申万行业
评级日期,,,,,,,,,,,,,,,
2009-01-31,000001,2009,平安银行,2009-01-31,BBB,82.56,B,72.98,BBB,80.26,A,88.73,货币金融服务,股份制银行,股份制银行Ⅲ


观察日期序列，为4个季度的最后一个自然日，非交易日

In [25]:
df_esg.index.unique()

DatetimeIndex(['2009-01-31', '2009-04-30', '2009-07-31', '2009-10-31',
               '2010-01-31', '2010-04-30', '2010-07-31', '2010-10-31',
               '2011-01-31', '2011-04-30', '2011-07-31', '2011-10-31',
               '2012-01-31', '2012-04-30', '2012-07-31', '2012-10-31',
               '2013-01-31', '2013-04-30', '2013-07-31', '2013-10-31',
               '2014-01-31', '2014-04-30', '2014-07-31', '2014-10-31',
               '2015-01-31', '2015-04-30', '2015-07-31', '2015-10-31',
               '2016-01-31', '2016-04-30', '2016-07-31', '2016-10-31',
               '2017-01-31', '2017-04-30', '2017-07-31', '2017-10-31',
               '2018-01-31', '2018-04-30', '2018-07-31', '2018-10-31',
               '2019-01-31', '2019-04-30', '2019-07-31', '2019-10-31',
               '2020-01-31', '2020-04-30', '2020-07-31', '2020-10-31',
               '2021-01-31', '2021-04-30', '2021-07-31', '2021-10-31',
               '2022-01-31', '2022-04-30', '2022-07-31', '2022-10-31',
      

In [27]:
processed_esg = df_esg.pivot(index='评级日期', columns='证券代码', values='综合评级')
processed_esg.head()

证券代码,000001,000002,000004,000005,000006,000007,000008,000009,000010,000011,...,873169,873223,873305,873339,873527,873576,873593,873665,873693,873726
评级日期,,,,,,,,,,,,,,,,,,,,,
2009-01-31,BBB,BBB,CCC,CCC,BB,B,B,B,C,BB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-04-30,BBB,BB,CCC,CCC,BBB,B,B,BB,CC,B,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-07-31,BBB,BB,CCC,CCC,BBB,B,B,BB,CC,B,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-10-31,BBB,BBB,CCC,CCC,BB,B,B,BB,CC,B,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-31,BBB,BBB,CC,CCC,BB,B,B,BB,CC,B,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Resampling

将价格的日频率数据转换为季度频率，让其index向esg数据（季度频率）靠齐

In [65]:
a = processed_esg.index
common_index = a.append(df_prices.index).unique()
processed_prices = df_prices.reindex(common_index)
processed_prices = processed_prices.sort_index()
processed_prices = processed_prices.ffill()

In [66]:
processed_prices = processed_prices.loc[processed_esg.index]

股票数向价格靠齐

In [72]:
processed_esg = processed_esg.loc[:, processed_prices.columns]

In [73]:
print(processed_esg.shape, processed_prices.shape)

(60, 2212) (60, 2212)


## Spillover effect computing

### Conditional volatility computing

将价格数据转换为收益率序列

In [4]:
import pandas as pd
import numpy as np

def calculate_log_returns(df):
    # 计算对数收益率
    return np.log(df / df.shift(1)).dropna()

In [76]:
log_returns = calculate_log_returns(processed_prices.ffill().dropna())
log_returns.head()

,000001,000002,000004,000005,000006,000007,000008,000009,000010,000011,...,601933,601939,601958,601988,601989,601991,601992,601996,601998,601999
评级日期,,,,,,,,,,,,,,,,,,,,,
2012-04-30,-0.004824,0.160294,0.030860,0.028234,0.369116,0.494034,0.219007,-0.148540,-0.207481,0.292603,...,0.097670,0.005391,0.107088,0.016667,0.092570,0.007240,-0.005872,0.023013,0.036589,-0.012136
2012-07-31,-0.092287,0.041357,-0.066567,-0.374537,-0.168344,0.363734,0.094145,-0.304430,0.118711,-0.343196,...,-0.447432,-0.134044,-0.164937,-0.056672,-0.262263,-0.024648,-0.345464,0.145209,-0.102924,-0.232651
2012-10-31,-0.130100,-0.103795,0.030823,0.060477,0.012114,-0.027633,0.148550,0.065893,0.083202,0.065284,...,0.222658,0.040166,-0.058622,0.000000,-0.029493,-0.203241,-0.035567,-0.297712,-0.095089,0.049597
2013-01-31,0.472088,0.367072,0.124433,0.066539,0.287099,0.128799,0.100055,0.103041,0.250924,0.161278,...,0.013501,0.184616,0.152861,0.133531,0.094692,0.057221,0.330296,0.534956,0.340981,-0.032790
2013-04-30,-0.120744,-0.085116,0.132468,-0.172087,-0.228575,-0.121319,0.106937,0.021371,0.244100,0.017634,...,-0.000767,-0.067708,-0.236786,-0.085158,-0.165143,0.007459,-0.201084,-0.036418,-0.163130,-0.130937


In [7]:
from arch import arch_model
from joblib import Parallel, delayed
from tqdm import tqdm

def estimate_volatility_single_series(returns):
    model = arch_model(returns, vol='Garch', p=1, q=1)
    res = model.fit(disp='off')
    return res.conditional_volatility

In [8]:
def estimate_volatility(returns, n_jobs=50):
    # 使用并行处理每列数据
    results = Parallel(n_jobs=n_jobs)(delayed(estimate_volatility_single_series)(returns[col]) for col in tqdm(returns.columns))
    
    # 将结果合并为一个 DataFrame
    volatilities = pd.concat(results, axis=1)
    volatilities.columns = returns.columns
    volatilities.index = returns.index
    
    return volatilities

In [9]:
volatilities = estimate_volatility(log_returns*100.)

  5%|▍         | 100/2212 [00:02<00:50, 41.65it/s]

KeyboardInterrupt: 

### DY method

使用Diebold和Yilmaz的方法计算波动率溢出指数。该方法通过VAR（向量自回归）模型计算不同时间序列之间的波动率传递和溢出效应。


In [2]:
conditional_vol = pd.read_csv('datasets/processed_data/conditional_vol.csv', index_col=0)
conditional_vol.head()

,000001,000002,000004,000005,000006,000007,000008,000009,000010,000011,...,601933,601939,601958,601988,601989,601991,601992,601996,601998,601999
time,,,,,,,,,,,,,,,,,,,,,
2012-01-20,1.884160,2.066665,2.286852,0.731061,2.407929,2.721594,2.209964,2.812903,1.363675,3.331488,...,2.755140,1.089197,2.288879,0.861095,2.136018,1.535771,2.686177,2.376037,1.324802,2.297640
2012-01-30,2.411903,2.031186,2.219985,0.857659,2.361746,2.543625,2.148125,2.714427,1.391026,3.227820,...,3.717847,1.035498,2.153772,0.865869,2.003292,1.517830,2.651255,2.366631,1.456511,2.256588
2012-01-31,2.336884,2.364132,2.151476,0.947898,2.544614,2.355691,2.019882,2.681453,1.411791,3.130959,...,3.557059,1.512826,2.166017,0.955502,1.916189,1.574921,2.729802,2.273397,1.514844,2.151353
2012-02-01,2.264754,2.294564,2.140024,1.015135,2.452450,2.392157,2.598155,2.720486,1.427624,2.961174,...,3.463173,1.405492,2.034719,0.910728,1.915663,1.611220,2.631289,2.190599,1.404144,2.633883
2012-02-02,2.212680,2.226720,2.171805,1.066499,2.386096,2.218796,2.411577,2.809333,1.439735,2.929225,...,3.264411,1.316867,2.437764,0.887648,1.941197,1.581820,2.943972,2.107233,1.480218,2.752529


In [ ]:
conditional_vol.

In [3]:
from statsmodels.tsa.api import VAR
model = VAR(conditional_vol)
results = model.fit(maxlags=5)
H=5

/Users/hans/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


In [4]:
from tqdm import tqdm
# 获取模型残差的协方差矩阵及其逆矩阵
sigma_u = results.sigma_u
sigma_u_inv = np.linalg.inv(sigma_u)

# 计算 GFEVD
def compute_gfevd(H, A, sigma_u, sigma_u_inv):
    n_vars = A.shape[1]
    gfevd = np.zeros((n_vars, n_vars))
    e = np.eye(n_vars)
    
    for i in tqdm(range(n_vars)):
        for j in range(n_vars):
            num = sigma_u_inv[j, j] * np.sum([(e[i].T @ A[h] @ sigma_u @ e[j]) ** 2 for h in range(H)]) 
            den = np.sum([(e[i].T @ A[h] @ sigma_u @ A[h].T @ e[i]) for h in range(H)])
            gfevd[i, j] = num / den
    return gfevd


In [5]:
MA_A = results.ma_rep(maxn=5)

In [6]:
# 计算 GFEVD
gfevd = compute_gfevd(H, MA_A, sigma_u, sigma_u_inv)

gfevd = pd.DataFrame(gfevd[:,:], index=volatilities.columns,
             columns=volatilities.columns)

gfevd

  2%|▏         | 42/2212 [2:31:02<130:04:04, 215.78s/it]


KeyboardInterrupt: 

In [ ]:
a = gfevd/10e14
plt.hist(a.values.ravel())

### Scaling

In [ ]:
def scale_one(df):
    # j 归一化
    s1 = df.values
    s2 = df.values.sum(axis=1)
    s3 = (s1.T/s2).T
    return pd.DataFrame(s3, index=df.index, columns=df.index)

In [ ]:
spillover_matrix = scale_one(gfevd)
spillover_matrix

In [ ]:
esg_quantiles = df_esg.groupby('月份')['综合得分'].quantile([0.2, 0.4, 0.6, 0.8]).unstack()
esg_quantiles.head(5)

In [ ]:
resample_prices = df_prices.reindex(pd.date_range(start=df_prices.index.min(),
                                         end=df_prices.index.max(),
                                         freq='D'))

In [ ]:
diff = np.log(resample_prices.resample('M').last() / resample_prices.resample('M').first())
diff.index = ['%s-%s' %(diff.index.year[i], diff.index.month[i]) for i in range(diff.shape[0])]
diff.head()

In [ ]:
# 为每个股票计算五分位
def assign_quantile(row):
    for quantile in [0.2, 0.4, 0.6, 0.8]:
        if row['综合得分'] <= esg_quantiles.loc[row['月份'], quantile]:
            return int(quantile * 5)
    return 5

df_esg['quantile'] = df_esg.apply(assign_quantile, axis=1)

In [ ]:
pivot_df = pd.DataFrame()
for i in tqdm(range(diff.shape[1])):
    df = pd.DataFrame()
    df['收益率'] = diff.iloc[:, i]
    df['证券代码'] = diff.columns[i]
    pivot_df = pd.concat([pivot_df, df])

In [ ]:
pivot_df

In [ ]:
ret = []
for i in tqdm(range(df_esg.shape[0])):
    if df_esg.iloc[i]['月份'] in diff.index and df_esg.iloc[i]['证券代码'] in diff.columns: 
        ret.append(diff.loc[df_esg.iloc[i]['月份'], df_esg.iloc[i]['证券代码']])
    else:
        ret.append(np.nan)

In [ ]:
df_esg['收益率'] = ret

In [ ]:
df_esg.head()

In [ ]:
a = df_esg.groupby(['quantile', '月份'])['收益率'].mean()

In [ ]:
import matplotlib.pyplot as plt
for i in [1,2,3,4,5]:
    a[i].index = pd.to_datetime(a[i].index)
    b = a[i].sort_index()
    b.plot()

In [ ]:
b

In [ ]:
from arch import arch_model
from joblib import Parallel, delayed
from tqdm import tqdm

def estimate_volatility_single_series(returns):
    model = arch_model(returns, vol='Garch', p=1, q=1)
    res = model.fit(disp='off')
    return res.conditional_volatility

In [ ]:
def estimate_volatility(returns, n_jobs=50):
    # 使用并行处理每列数据
    results = Parallel(n_jobs=n_jobs)(delayed(estimate_volatility_single_series)(returns[col]) for col in tqdm(returns.columns))
    
    # 将结果合并为一个 DataFrame
    volatilities = pd.concat(results, axis=1)
    volatilities.columns = returns.columns
    volatilities.index = returns.index
    
    return volatilities

In [ ]:
a[1].pd.to_datetime(a[1].index)

In [ ]:
volatilities = estimate_volatility(a[1]*100.)

### 数据平稳性检验和差分处理

In [ ]:
from statsmodels.tsa.stattools import adfuller
from tqdm.notebook import tqdm
adf_no_list = []
def check_stationarity(series):
    result = adfuller(series)
    return result[1]  # 返回p值


for col in tqdm(volatilities.columns):
    p_value = check_stationarity(volatilities[col])
    if p_value > 0.05:            # 如果p值大于0.05，进行差分
        adf_no_list.append(col)
print(len(adf_no_list))

In [ ]:
print(len(adf_no_list))

### 波动率溢出指数

使用Diebold和Yilmaz的方法计算波动率溢出指数。该方法通过VAR（向量自回归）模型计算不同时间序列之间的波动率传递和溢出效应。


In [ ]:
volatilities = volatilities.drop(adf_no_list, axis=1)
volatilities.shape

In [ ]:
# 查找全是常数的列
constant_columns = [col for col in volatilities.columns if volatilities[col].nunique() < 100]
constant_columns

In [ ]:
# 删除全是常数的列
volatilities = volatilities.drop(columns=constant_columns)

In [ ]:
volatilities = volatilities.iloc[:252, :300]
volatilities.to_csv('datasets/processed_data/volatilities.csv')

In [ ]:
from statsmodels.tsa.api import VAR
model = VAR(volatilities)
results = model.fit(maxlags=5)
H=5

In [ ]:
from tqdm import tqdm
# 获取模型残差的协方差矩阵及其逆矩阵
sigma_u = results.sigma_u
sigma_u_inv = np.linalg.inv(sigma_u)

# 计算 GFEVD
def compute_gfevd(H, A, sigma_u, sigma_u_inv):
    n_vars = A.shape[1]
    gfevd = np.zeros((n_vars, n_vars))
    e = np.eye(n_vars)
    
    for i in tqdm(range(n_vars)):
        for j in range(n_vars):
            num = sigma_u_inv[j, j] * np.sum([(e[i].T @ A[h] @ sigma_u @ e[j]) ** 2 for h in range(H)]) 
            den = np.sum([(e[i].T @ A[h] @ sigma_u @ A[h].T @ e[i]) for h in range(H)])
            gfevd[i, j] = num / den
    return gfevd


In [ ]:
MA_A = results.ma_rep(maxn=5)

In [ ]:
# 计算 GFEVD
gfevd = compute_gfevd(H, MA_A, sigma_u, sigma_u_inv)

gfevd = pd.DataFrame(gfevd[:,:], index=volatilities.columns,
             columns=volatilities.columns)

gfevd

In [ ]:
a = gfevd/10e14
plt.hist(a.values.ravel())

### 归一化

In [ ]:
def scale_one(df):
    # j 归一化
    s1 = df.values
    s2 = df.values.sum(axis=1)
    s3 = (s1.T/s2).T
    return pd.DataFrame(s3, index=df.index, columns=df.index)

In [ ]:
spillover_matrix = scale_one(gfevd)
spillover_matrix

In [ ]:
import pandas as pd
path='datasets/华证2009-2023年（含细分项+季度)）/华证esg评级2009-2023（细分项）/华证esg评级含细分项（年度）2009-2023.xlsx'
ESG_df = pd.read_excel(path, dtype={'股票代码':str})
ESG_df['股票代码'] = ESG_df['股票代码'].str.zfill(6)
ESG_df.head(1)

In [ ]:
ESG = ESG_df[['股票代码', '综合得分']].groupby('股票代码').mean()

In [ ]:
df = pd.DataFrame((gfevd/10e14).sum(axis=0), columns=['spillover'])
df['ESG_Score'] = ESG
df

In [ ]:
df.plot(kind='scatter', x='ESG_Score', y='spillover')

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
import statsmodels.api as sm

In [ ]:
# 计算皮尔逊相关系数
corr, p_value = pearsonr(df['spillover'], df['ESG_Score'])
print(f"Pearson correlation coefficient: {corr:.4f}, P-value: {p_value:.4f}")

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
import statsmodels.api as sm

X = df['spillover']
y = df['ESG_Score']
X = sm.add_constant(X)  # 添加常数项
model = sm.OLS(y, X).fit()
print(model.summary())

### 净溢出

检查是否需要标准化

In [ ]:
spillover_matrix.sum().sum() 

In [ ]:
net_spillover_matrix = spillover_matrix.copy() * 0.
for i in spillover_matrix.index:
    for j in spillover_matrix.columns:
        net = spillover_matrix.loc[i,j] - spillover_matrix.loc[j,i]
        if net >= 0:
            net_spillover_matrix.loc[i,j] = net
        else:
            net_spillover_matrix.loc[j,i] = -1 * net

### 热力图

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib_inline import backend_inline
backend_inline.set_matplotlib_formats('svg') 

sns.heatmap(net_spillover_matrix, annot=True, fmt=".3f", cmap="YlGnBu")

plt.title("Net Spillover Matrices")

plt.show()


In [ ]:
import pandas as pd
net_spillover_matrix = pd.read_csv('bb.csv', index_col=0, dtype={'code':str})
net_spillover_matrix = net_spillover_matrix.iloc[:100, :100]

## 构建溢出网络

根据计算得到的溢出指数，构建各个股票网络之间的溢出矩阵。


将溢出矩阵转换为图网络，节点代表股票，边的权重代表波动率溢出的强度。


In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

def build_spillover_network(spillover_matrix):
    G = nx.DiGraph()
    num_nodes = spillover_matrix.shape[0]
    for i in tqdm(spillover_matrix.index):
        for j in spillover_matrix.columns:
            if i != j:
                G.add_edge(i, j, weight=spillover_matrix.loc[i, j])
    return G

G = build_spillover_network(net_spillover_matrix)


In [ ]:
import seaborn as sns
# 使用Seaborn的调色板生成九种颜色
palette = sns.color_palette("husl", 9)
# 转换为十六进制颜色代码
colors_list = [sns.desaturate(color, 0.8) for color in palette]

In [ ]:
# import matplotlib.patches as mpatches
# ESG_rate = np.array(['AAA', 'AA', 'A', 'BBB', 'BB', 'B', 'CCC', 'CC', 'C'])
# node_colors = []
# used_rate = set()
# for code in net_spillover_matrix.index:
#     a = ESG_df.loc[ESG_df['证券代码'] == str(code), 'ESG评级'].values[0]
#     loc = np.where(ESG_rate == str(a))[0][0]
#     node_colors.append(colors_list[loc])
#     used_rate.add(loc)
# legend_color = []
# for c in used_rate:
#     legend_color.append(mpatches.Patch(color=colors_list[c], label=ESG_rate[c]))

In [ ]:
import numpy as np
# 定义边的权重
edge_weights = []
for u, v in G.edges():
    if G[u][v]['weight'] == 0:
        edge_weights.append(np.nan)
    else:
        if np.abs(G[u][v]['weight']*20 )/10>10:
            edge_weights.append(np.abs(G[u][v]['weight']*20 )/10)
        else:
            edge_weights.append(np.nan)

In [ ]:
(~np.isnan(edge_weights)).sum()

In [ ]:
import matplotlib.cm as cm
plt.figure(figsize=(6, 6))  
pos = nx.spring_layout(G)  # 使用弹簧布局

#node_style = { 'node_size': 100, 'alpha': 0.8} #'node_color': node_colors,
edge_style = {'width': edge_weights, 'alpha': 0.5} #'edge_color': 'gray', 
nx.draw(G,
        pos=pos,
        with_labels=False, 
        font_size=6,
        font_color='black',
        font_weight='bold',
        arrows=True,
        arrowsize=8,
        #**node_style,
        **edge_style
       )

#plt.legend(handles=legend_color, loc='best')

In [ ]:
sns.clustermap(net_spillover_matrix)